In [1]:
from keras import backend as K
K.set_image_data_format('channels_first')
import os
import numpy as np
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 


Using TensorFlow backend.


In [2]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    #Compute distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    #Compute distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    #subtract the two previous distances and add alpha.
    basic_loss = pos_dist-neg_dist+alpha
    #Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))

    
    return loss

In [3]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [4]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [5]:
database = {}
database["pramod"] = img_path_to_encoding("images/pramod.jpg", FRmodel)
database["barath"] = img_path_to_encoding("images/barath.jpg", FRmodel)
database["rittwik"] = img_path_to_encoding("images/rittwik.jpg", FRmodel)
database["shatad"] = img_path_to_encoding("images/shatad.jpg", FRmodel)

In [8]:
def verify(image_path, identity, database, model):
    encoding = img_path_to_encoding(image_path,model)
    
    dist = np.linalg.norm(database[identity]-encoding)
    
    if dist<0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
    return dist, door_open

In [9]:
verify("images - Copy/test6.jpg", "pramod", database, FRmodel)

It's pramod, welcome home!


(0.5410353, True)

In [10]:
def who_is_it(image_path, database, model):
    encoding = img_path_to_encoding(image_path,model)
    min_dist = 100
    
    for (name, db_enc) in database.items():    
        dist = np.linalg.norm(db_enc - encoding)
        if dist<=min_dist:
            min_dist = dist
            identity = name

    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [12]:
who_is_it("images - Copy/test8.jpg", database, FRmodel)

it's barath, the distance is 0.533336


(0.533336, 'barath')